# MCMC Analysis for Sedimentary Sequences

This notebook demonstrates the use of Markov Chain Monte Carlo methods to:
1. Block well logs using continuous wavelet transform
2. Cluster facies using Self-Organizing Maps (SOM)
3. Build transition probability matrices from observed sequences
4. Generate synthetic logs via Markov chains
5. Correlate logs using Dynamic Time Warping (DTW)
6. Assess correlation significance through Monte Carlo simulation

**Author:** Zoltan Sylvester

## Imports and Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import the sedimentary MCMC module with all analysis functions
import sedimentary_mcmc as sedmc

# Graphics configuration
%matplotlib qt
plt.rcParams['svg.fonttype'] = 'none'

## Step 1: Load Wheeler Diagram Images and Convert to Logs

Load Wheeler diagram images for three cores/locations. These images represent stratigraphic surfaces in Wheeler (chronostratigraphic) space.

The images are converted to 1D logs by averaging across columns. The red channel is used, and values are inverted (darker = higher values).

In [ ]:
im_wheeler_peterson = plt.imread('data/peterson_wheeler_new.png')
im_wheeler_5412 = plt.imread('data/core_5412_wheeler_new.png')
im_wheeler_4431 = plt.imread('data/core_4431_wheeler_new.png')

peterson_log = sedmc.image_to_log(im_wheeler_peterson, channel=0, invert=True, trim_start=1)
core_5412_log = sedmc.image_to_log(im_wheeler_5412, channel=0, invert=True, trim_end=2)
core_4431_log = sedmc.image_to_log(im_wheeler_4431, channel=0, invert=True)

print(f"Peterson log length: {len(peterson_log)}")
print(f"Core 5412 log length: {len(core_5412_log)}")
print(f"Core 4431 log length: {len(core_4431_log)}")

## Step 2: Block the Logs Using Continuous Wavelet Transform

Log blocking identifies natural boundaries in the signal using the Continuous Wavelet Transform (CWT) and computes mean values within each block.

The `scale` parameter controls the granularity of blocking:
- **Lower scale** = more boundaries (finer blocks)
- **Higher scale** = fewer boundaries (coarser blocks)

In [ ]:
scale = 1  # Fine-scale blocking

# Block each log - this creates discrete intervals with mean property values
peterson_md, peterson_blocked_log, peterson_tops, peterson_props = sedmc.log_blocking(peterson_log, scale)
core_5412_md, core_5412_blocked_log, core_5412_tops, core_5412_props = sedmc.log_blocking(core_5412_log, scale)
core_4431_md, core_4431_blocked_log, core_4431_tops, core_4431_props = sedmc.log_blocking(core_4431_log, scale)

# Remove any leading NaN values from Peterson log
peterson_tops = peterson_tops[1:]
peterson_props = peterson_props[1:]

print(f"Peterson: {len(peterson_props)} blocks")
print(f"Core 5412: {len(core_5412_props)} blocks")
print(f"Core 4431: {len(core_4431_props)} blocks")

## Step 3: Cluster Facies Using Self-Organizing Map (SOM)

Combine data from all cores to train a SOM for facies classification. Each facies is characterized by:
- **vsh**: volume of shale (proxy for lithology)
- **ths**: thickness of the interval

The SOM is configured with:
- **PCA initialization**: Aligns weights along principal components for better topological organization
- **Larger sigma**: Creates more global organization initially
- **More iterations**: Ensures better convergence

In [ ]:
# Combine property values and thicknesses from all cores
vsh = np.hstack((peterson_props, core_5412_props, core_4431_props))
ths = np.hstack((np.diff(peterson_tops), np.diff(core_5412_tops), np.diff(core_4431_tops)))

# Cluster the facies data using SOM (3x3 = 9 possible clusters)
som_shape = (4, 4)
cluster_index, som, data_normalized = sedmc.cluster_facies_data(
    vsh, ths, 
    som_shape=som_shape, 
    sigma=1.0,
    learning_rate=0.25, 
    n_iterations=10000,
    init_method='pca',
    random_seed=42,
    plot=False
)

# Generate 2D colormap for SOM visualization
colormap, colormap_2d = sedmc.generate_2d_colormap(som_shape, method='corners')

# Plot the clusters with 2D colormap and neighbor links
fig, axes = sedmc.plot_som_clusters(
    data_normalized, cluster_index, som, colormap, colormap_2d, som_shape,
    show_colormap_reference=True, show_neighbor_links=True
)

## Step 4: Build Transition Probability Matrix

The transition matrix captures the probability of transitioning between different facies clusters. This is the core of the Markov chain model - it defines how likely we are to move from one facies type to another in the stratigraphic sequence.

In [ ]:
# Build transition matrix from the observed cluster sequence
M = sedmc.transition_matrix(cluster_index[::-1])

# Visualize the transition probability matrix with color-coded cluster labels
fig, axes, cbar = sedmc.plot_transition_matrix(M, colormap, som_shape, colormap_2d=colormap_2d)

## Step 5: Generate Synthetic Log and Test Correlation

Use the Markov chain to generate a synthetic facies sequence, then correlate it with a real log using Dynamic Time Warping (DTW).

**Parameters:**
- `chain_length`: Number of facies in the synthetic sequence
- `exponent`: Cost function exponent for DTW (lower = more sensitive to small differences)

In [ ]:
chain_length = 370  # Number of facies in the synthetic sequence
exponent = 0.15     # Cost function exponent for DTW

# Generate a synthetic facies sequence
vsh_chain, ths_chain = sedmc.facies_chain(M, vsh, ths, cluster_index, chain_length)

# Plot the synthetic facies column
fig1, ax1 = plt.subplots(figsize=(3, 10))
sedmc.plot_facies_column(vsh_chain, ths_chain, ax=ax1)
ax1.set_title('Synthetic Facies Column')

# Convert chain to a blocked log for correlation
md_synth, grn_synth, depths_synth = sedmc.chain_to_log(vsh_chain, ths_chain)

# Correlate with Core 5412 log
log1 = core_5412_blocked_log
log2 = grn_synth
p, q, D = sedmc.correlate_logs(log1, log2, exponent)

# Plot the DTW cost matrix with warping path
fig2, ax2 = plt.subplots(figsize=(8, 6))
sedmc.plot_correlation_matrix(D, p, q, ax=ax2)
ax2.set_xlabel('Synthetic Log Index')
ax2.set_ylabel('Core 5412 Log Index')
ax2.set_title('DTW Cost Matrix with Warping Path')

# Calculate correlation quality
r_value, slope, intercept = sedmc.compute_correlation_quality(log1, log2, p, q)
print(f"Correlation coefficient: {r_value:.4f}")
print(f"Path deviation: {np.sum(np.abs(p-q))}")

## Step 6: Monte Carlo Simulation for Correlation Significance

Generate many synthetic logs and correlate each with the Peterson log. This builds a **null distribution** of correlation coefficients to assess whether the real Peterson-4431 correlation is statistically significant.

If the real correlation is significantly higher than the synthetic correlations, it suggests the two cores share a genuine stratigraphic signal beyond what would be expected by chance.

In [ ]:
# Monte Carlo parameters
n_simulations = 1000
chain_length = 370
exponent = 0.15
noise_std = 0.01  # Small amount of noise added to logs

# Run Monte Carlo simulation
print("Running Monte Carlo simulation...")
r_values, log_lengths = sedmc.run_monte_carlo_correlation(
    reference_log=peterson_blocked_log,
    M=M,
    vsh=vsh,
    ths=ths,
    cluster_index=cluster_index,
    n_simulations=n_simulations,
    chain_length=chain_length,
    exponent=exponent,
    noise_std=noise_std
)

# Calculate the real Peterson-4431 correlation for comparison
log1 = peterson_blocked_log + np.random.normal(0, noise_std, len(peterson_blocked_log))
log2 = core_4431_blocked_log
p, q, D = sedmc.correlate_logs(log1, log2, exponent)
real_r_value, _, _ = sedmc.compute_correlation_quality(log1, log2, p, q)

# Plot results with the real correlation marked
sedmc.plot_monte_carlo_results(r_values, comparison_r_value=real_r_value, n_bins=50)
plt.xlabel('Correlation Coefficient (r)')
plt.ylabel('Count')
plt.title('Monte Carlo Correlation Distribution\n(Red line = Peterson-4431 correlation)')

# Calculate p-value (proportion of synthetic correlations >= real correlation)
p_value = np.sum(np.array(r_values) >= real_r_value) / len(r_values)
print(f"\nReal Peterson-4431 correlation: {real_r_value:.4f}")
print(f"Mean synthetic correlation: {np.mean(r_values):.4f}")
print(f"Std synthetic correlation: {np.std(r_values):.4f}")
print(f"Empirical p-value: {p_value:.4f}")